In [2]:
import pandas as pd

complex_df = pd.read_pickle('complex_label.pkl')

In [3]:
complex_df.columns

Index(['GeneAB', 'Same_Complex'], dtype='object')

In [4]:
pw = pd.read_pickle('allpairs_spearman_correlation.pkl')

In [5]:
import os
df=pd.DataFrame()
for f in os.listdir('scgpt_split_outputs'):
    if f.startswith('scGPT'):
        
        name = f[:-20]
        print(f'Loading {name}...')
        data = pd.read_pickle('scgpt_split_outputs/' + f)
        df[name] = data['Cosine_Similarity'].to_list()

Loading scGPT_bc_embeddings_similarities...
Loading scGPT_pancancer_embeddings_similarities...
Loading scGPT_CP_embeddings_similarities...
Loading scGPT_lung_embeddings_similarities...
Loading scGPT_heart_embeddings_similarities...
Loading scGPT_brain_embeddings_similarities...
Loading scGPT_kidney_embeddings_similarities...
Loading scGPT_human_embeddings_similarities...


In [6]:
for f in os.listdir('gf_split_outputs'):
    if f.startswith('GF'):
        
        name = f[:-20]
        print(f'Loading {name}...')
        data = pd.read_pickle('gf_split_outputs/' + f)
        df[name] = data['Cosine_Similarity'].to_list()

Loading GF-6L30M_HUMANemb_similarities...
Loading GF-20L95M_HUMANemb_similarities...
Loading GF-12L95M_HUMANemb_similarities...
Loading GF-12L95MCANCER_UNIPROT_HUMANemb_similarities...
Loading GF-12L30M_HUMANemb_similarities...


In [7]:
df['Correlation'] = pw['Correlation'].to_list()

In [8]:
df['Same_Complex'] = complex_df['Same_Complex'].to_list()

In [9]:
# df['Gene_A'] = gene_pairs['Gene_A'].to_list()
# df['Gene_B'] = gene_pairs['Gene_B'].to_list()

df['GeneAB'] = complex_df['GeneAB'].to_list()

In [10]:
df

,scGPT_bc_embeddings_similarities,scGPT_pancancer_embeddings_similarities,scGPT_CP_embeddings_similarities,scGPT_lung_embeddings_similarities,scGPT_heart_embeddings_similarities,scGPT_brain_embeddings_similarities,scGPT_kidney_embeddings_similarities,scGPT_human_embeddings_similarities,GF-6L30M_HUMANemb_similarities,GF-20L95M_HUMANemb_similarities,GF-12L95M_HUMANemb_similarities,GF-12L95MCANCER_UNIPROT_HUMANemb_similarities,GF-12L30M_HUMANemb_similarities,Correlation,Same_Complex,GeneAB
0,NaN,0.566069,0.356636,0.224554,0.117288,0.114432,0.149671,0.357125,NaN,NaN,NaN,NaN,NaN,0.395247,0,ST13P4_WASH2P
1,NaN,-0.083783,0.105390,0.051439,0.034904,0.051954,0.024855,0.106240,NaN,NaN,NaN,NaN,NaN,0.121127,0,ST13P4_FAM90A12P
2,NaN,0.336766,0.159616,0.118687,0.068871,0.069045,0.060269,0.160441,NaN,NaN,NaN,NaN,NaN,0.359050,0,ST13P4_PI4KAP1
3,NaN,0.258465,0.210777,0.104398,0.070150,0.016221,0.079906,0.211458,NaN,NaN,NaN,NaN,NaN,0.273463,0,ST13P4_SNX29P2
4,NaN,0.151502,0.099180,-0.046862,-0.016629,0.038130,-0.063625,0.100117,NaN,NaN,NaN,NaN,NaN,0.378892,0,ST13P4_ZNF818P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200794870,NaN,0.058843,-0.123608,0.025317,0.011302,0.020572,0.028625,-0.122569,NaN,NaN,NaN,NaN,NaN,NaN,0,RBMXP1_PPT1
200794871,NaN,0.055566,0.068872,0.036179,-0.039677,-0.038647,-0.015523,0.069859,NaN,NaN,NaN,NaN,NaN,NaN,0,RBMXP1_PLD2
200794872,NaN,0.012171,-0.037853,-0.023328,-0.022463,-0.078864,-0.025873,-0.037521,NaN,NaN,NaN,NaN,NaN,NaN,0,RBMXP1_EIF4EBP1
200794873,NaN,0.020594,0.128618,0.060834,0.002627,-0.001737,0.032308,0.135589,NaN,NaN,NaN,NaN,NaN,NaN,0,RBMXP1_TINAG


In [21]:
df_dropna = df.dropna()

In [27]:
df

,scGPT_bc_embeddings_similarities,scGPT_pancancer_embeddings_similarities,scGPT_CP_embeddings_similarities,scGPT_lung_embeddings_similarities,scGPT_heart_embeddings_similarities,scGPT_brain_embeddings_similarities,scGPT_kidney_embeddings_similarities,scGPT_human_embeddings_similarities,GF-6L30M_HUMANemb_similarities,GF-20L95M_HUMANemb_similarities,GF-12L95M_HUMANemb_similarities,GF-12L95MCANCER_UNIPROT_HUMANemb_similarities,GF-12L30M_HUMANemb_similarities,Correlation,Same_Complex,Gene_A,Gene_B
0,NaN,0.566069,0.356636,0.224554,0.117288,0.114432,0.149671,0.357125,NaN,NaN,NaN,NaN,NaN,0.395247,0,ST13P4,WASH2P
1,NaN,-0.083783,0.105390,0.051439,0.034904,0.051954,0.024855,0.106240,NaN,NaN,NaN,NaN,NaN,0.121127,0,ST13P4,FAM90A12P
2,NaN,0.336766,0.159616,0.118687,0.068871,0.069045,0.060269,0.160441,NaN,NaN,NaN,NaN,NaN,0.359050,0,ST13P4,PI4KAP1
3,NaN,0.258465,0.210777,0.104398,0.070150,0.016221,0.079906,0.211458,NaN,NaN,NaN,NaN,NaN,0.273463,0,ST13P4,SNX29P2
4,NaN,0.151502,0.099180,-0.046862,-0.016629,0.038130,-0.063625,0.100117,NaN,NaN,NaN,NaN,NaN,0.378892,0,ST13P4,ZNF818P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200794870,NaN,0.058843,-0.123608,0.025317,0.011302,0.020572,0.028625,-0.122569,NaN,NaN,NaN,NaN,NaN,NaN,0,RBMXP1,PPT1
200794871,NaN,0.055566,0.068872,0.036179,-0.039677,-0.038647,-0.015523,0.069859,NaN,NaN,NaN,NaN,NaN,NaN,0,RBMXP1,PLD2
200794872,NaN,0.012171,-0.037853,-0.023328,-0.022463,-0.078864,-0.025873,-0.037521,NaN,NaN,NaN,NaN,NaN,NaN,0,RBMXP1,EIF4EBP1
200794873,NaN,0.020594,0.128618,0.060834,0.002627,-0.001737,0.032308,0.135589,NaN,NaN,NaN,NaN,NaN,NaN,0,RBMXP1,TINAG


In [22]:
df_dropna.to_pickle('full_dataset_drop_na.pkl')

In [11]:
df.to_pickle('full_dataset.pkl')